<a href="https://colab.research.google.com/github/ImHungry48/Next-Word-Prediction/blob/main/Next_Word_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
from random import choices
from enum import Enum

class SentenceMutator():
  class TYPO_TYPE(Enum):
      SWAP = 1
      SUBSTITUTE = 2
      DELETE = 3
      INSERT = 4
      CASE = 5
  
  def __init__(self, texts, typo_chance=0.005):
    # The chance of an individual character in a word being mutated
    self.typo_chance = typo_chance

    # All neighboring keys on a standard QWERTY keyboard
    self.neighboring_keys = {
        'q': 'was',
        'w': 'qeasd',
        'e': 'wrsdf',
        'r': 'etdfg',
        't': 'ryfgh',
        'y': 'tughj',
        'u': 'yihjk',
        'i': 'uojkl',
        'o': 'ipkl',
        'p': 'ol',
        'a': 'qwszx',
        's': 'qwesz',
        'd': 'wersfc',
        'f': 'ertdgcv',
        'g': 'rtyfhvb',
        'h': 'tyugjbn',
        'j': 'yuihknm',
        'k': 'uiojlm',
        'l': 'iopk',
        'z': 'asx',
        'x': 'sdzc',
        'c': 'dfxv',
        'v': 'fgcb',
        'b': 'vghn',
        'n': 'bhjm',
        'm': 'njk',
        '.': 'l;/\',',
        '?': ';\'.',
        '!': '12@wq`',
        '-': '=[p0]'
      }

    self.word_as_list = []
  
  def mutate_sentence(self, sentence):
    # Split the sentence into individual words
    words = sentence.split()

    # Create an array to store transformed words
    typoed_words = []

    # Mutate and store each word
    for word in words:
      typoed_words.append(mutate(word))
    
    # Return the sentence
    return ' '.join(typoed_words)

  def get_neighboring_keys(self, char, is_upper):
    # Get all neighboring keys of the given character
    neighbors = self.neighboring_keys[char]

    # Return a random neighboring key
    return random.choice(neighbors).upper() if is_upper else random.choice(neighbors)
  
  def swap(self, char_index, word):
    # Determine index of character is being swapped
    index_to_swap = char_index + 1

    if char_index == len(word) - 1: # If the last letter
      index_to_swap = len(word) - 2

    # Perform the swap
    temp = self.word_as_list[char_index]
    self.word_as_list[char_index] = self.word_as_list[index_to_swap]
    self.word_as_list[index_to_swap] = temp

  def substitute(self, char, char_index):
    # 80% chance to substitute with a neighboring character
    sub_with_nearby = random.random() < 0.8

    if sub_with_nearby:
      # Get neighboring keys
      sub_char = self.get_neighboring_keys(char.lower(), char.isupper())
      
    else:
      # Get a random letter in the alphabet
      characters = 'abcdefghijklmnopqrstuvwxyz!-=;,.\'\\?'
      sub_char = random.choice(list(characters))

    # Perform the substitution
    self.word_as_list[char_index] = sub_char

  def delete(self, char_index): 
    # Perform a soft deletion
    self.word_as_list[char_index] = -1

  def insert(self, char_index, char):
    # 80% chance to insert with a neighboring character
    insert_with_nearby = random.random() < 0.8

    if insert_with_nearby:
      # Get neighboring keys
      insert_char = self.get_neighboring_keys(char.lower(), char.isupper())
    else:
      # Get a random letter in the alphabet
      characters = 'abcdefghijklmnopqrstuvwxyz!-=;,.\'\\?'
      insert_char = random.choice(list(characters))

    # Perform the insertion
    self.word_as_list.insert(char_index, insert_char)

  def case(self, char, char_index):
    # Change letter to opposite case
    if char.isupper():
      self.word_as_list[char_index] = char.lower()
    else:
      self.word_as_list[char_index] = char.upper()

  def mutate(self, word):
    """ Mutates a single word by swapping, substituting, deleting, inserting, 
        or changing the case of characters.

    Args:
        word(str): The word to mutate.
    Returns:
        mutated_word(str): The word after ungoing mutations. Value does not change
        if not mutations occur
    """
    
    # Make the word a list for easier manipulation
    word_as_list = list(word)

    for char_index in range(len(word)):    
      char = word[char_index]    
      if random.random() <= self.typo_chance:
        # Select a random typo type
        typo_type= random.choices(list(self.TYPO_TYPE),
                              weights = [3, 4, 3, 4, 1])[0]
        
        # Implement the randomly selected typo type
        if typo_type == self.TYPO_TYPE.SWAP:

          # Only swap if there are enough characters in the word
          if len(word) < 2:
            break

          # Perform the swap
          self.swap(char_index)

        elif typo_type == self.TYPO_TYPE.SUBSTITUTE:
          # Perform the substitution
          self.substitute(char_index)

        elif typo_type == self.TYPO_TYPE.DELETE:
          # Only delete if there are enough characters in the word
          if len(word) < 2:
            break

          # Perform the deletion
          self.delete(char_index)

        elif typo_type == self.TYPO_TYPE.INSERT:
          # Perform the insertion
          self.insert(char_index, char)

        elif typo_type == self.TYPO_TYPE.CASE.value:
          # Perform the case change
          self.case(char, char_index)

    mutated_word = ''.join(char for char in word_as_list if type(char) == str)
    return mutated_word

In [ ]:
from nltk.tokenize import sent_tokenize
from torch.utils.data import Dataset

class SentenceDataset(Dataset):
  def __init__(self, texts, tokenizer, max_length, augmentation=False, typo_chance=0.005):
    self.tokenizer = tokenizer
    self.tokenized_sentences = []
    self.augmentation = augmentation
    self.mutator = SentenceMutator(texts, typo_chance)

    # Initalize the sentences in the dataset
    self.init_sentences(texts, typo_chance)
  
  def __len__(self):
    return len(self.tokenized_sentences)

  def __getitem__(self, idx):
    sentence = self.tokenized_sentences[idx]
    result = torch.tensor(sentence)
    return result

  def init_sentences(self, texts, typo_chance):
    for text in texts:
      for sentence in sent_tokenize(text):
        if self.augmentation:
          sentence = self.mutator(texts, typo_chance)
        sentence_tokens = tokenizer.encode(sentence, max_length=max_length, truncation=True)
        self.tokenized_sentences.append(sentence_tokens)